# Chatbot con un LLM utilizando Vector Stores - Interfaces de conversación 

En este cuaderno, vamos a adicionar funcionalidades que permitan enriquecer el contexto de los chatbot con Vector Stores

## Introducción

Una de las funcionalidades mas importantes cuando se construye un Chatbot es la capacidad de mantener el contexto de una conversación. Es importante que el LLM pueda mantener el hilo de la conversación.  

## Tokens 

Son unidades de texto o codigo fuente que los LLMs utilizan para procesar y generar respuestas. Los Token pueden ser de diferentes tipos como: 

1. Palabras
3. Caracteres
4. Otros segmentos de texto o codigo

Los token dependen entonces del metodo de tokenización utilizado. Se asigna un valor numerico o identificador y son organizados en secuencias o VECTORES que alimentan a los modelos y que de igual forma son su salida. 


## Embebidos (Embeddings) 

Los Embebidos son representaciones vectoriales de las palabras o tokens que capturan su significado semantico en un espacio dimensional ...... XD 
Mejor ...
Los embebidos son la representación o codificación de los Tokens; algunos ejemplos pueden ser sentencias, parrafos o documentos completos puestos en un espacio vectorial de muchas dimensiones donde cada una de estas dimensiones representa una caracteristica o atributo aprendido del lenguaje. 

Son entonces, la forma en la cual los LLMs comprenden el significado y las relaciones que existen en el lenguaje. 

## FAISS 

Es una libreria que nos permite realizar busquedas por similitud o agrupamiento utilizando vectores de muchas dimensiones. En otras palabras FAISS va a permitir realizar busquedas sobre los Embebidos generados para este tutorial. 

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS

## Dataset
En el directorio data vas a encontrar un documento llamado constitucion-politica-col.pdf, en el vamos a encontrar tan solo 2 paginas con los 10 primeros artículos. 

## Nota 
Langchain ofrece diferentes herramientas para cargar datos o documentos, puedes dar un vistazo en el siguiente enlace 
[LangChainDocumentLoaders](https://python.langchain.com/docs/modules/data_connection/document_loaders)

In [2]:
loader = PyPDFLoader("data/constitucion-politica-col.pdf")
documents_col = loader.load() #
print(f"Documentos cargados={len(documents_col)}")

Documentos cargados=2


Una vez hemos cargado los archivos debemos realizar una división del contenido para generar nuestros embeddings.
No podemos cargar un solo cuerpo de texto para generar los embeddings, debemos separar el texto en pequeños fragmentos que nuestro LLM pueda soportar 

In [3]:
docs = RecursiveCharacterTextSplitter(
    length_function = len,
    is_separator_regex = False,
    chunk_size=200, 
    chunk_overlap=40
).split_documents(documents_col)

Vemos que Langchain nos ha ayudado a dividir nuestro gran cuerpo de texto en frases mas cortas que podemos enviar a nuestro generador de Embeddings

In [4]:
print(f"Documentos generados={len(docs)}")

Documentos generados=23


## Instrucciones 

1. Alguien debe generar nuestros embeddigs, en nuestro ejemplo vamos a utilizar AWS Bedrock para generarlos y para lograrlo debemos conectarnos con el API de AWS
2. Vamos a almacenar en memoria los datos generados

In [6]:
vectorstore_faiss_aws = None
try:
    bedrock_embeddings = BedrockEmbeddings(
        credentials_profile_name="default", region_name="us-east-1"
    )
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = bedrock_embeddings
    )
    print(f"vectorstore faiss con aws bedrock: elementos en el indice={vectorstore_faiss_aws.index.ntotal}")

except ValueError as error:
    print(f"Error conectando con el vector store: {error}")
    raise error

vectorstore faiss con aws bedrock: elementos en el indice=23


Hagamos una consulta sobre nuestro Vector Store con una busqueda por similitud

In [11]:
query = "Artículo 5"
retrieved_docs = vectorstore_faiss_aws.similarity_search(query)
print(f" Documentos obtenidos: {len(retrieved_docs)}")

 Documentos obtenidos: 4


In [ ]:
## TODO: semantic search 